In [ ]:
from qdrant_haystack import QdrantDocumentStore
from haystack.nodes import EmbeddingRetriever
from haystack.nodes import PromptNode, PromptTemplate, AnswerParser
from haystack.pipelines import Pipeline
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer

In [ ]:
document_store = QdrantDocumentStore(
    "localhost",
    port = 6333,
    index="test_data",
    content_field = "content",
    name_field = "name",
    embedding_field = "vector",
    embedding_dim=384,
    hnsw_config={"m": 16, "ef_construct": 64},
    on_disk_payload = True
)

In [ ]:
model = "tiiuae/falcon-7b"

tokenizer = AutoTokenizer.from_pretrained(model)
model = AutoModelForCausalLM.from_pretrained(model, trust_remote_code=True)

In [ ]:
retriever = EmbeddingRetriever(
    document_store=document_store,
    embedding_model="sentence-transformers/multi-qa-MiniLM-L6-cos-v1",
    model_format="sentence_transformers",
    top_k=2
)

In [ ]:
rag_prompt = PromptTemplate(
    prompt="""Answer the following question \n\n Related text: {join(documents)} \n\n Question: {query} \n\n Answer:""",
    output_parser=AnswerParser())

prompt_node = PromptNode(model, model_kwargs={"model":model, "tokenizer": tokenizer}, default_prompt_template=rag_prompt)

In [ ]:
pipe = Pipeline()
pipe.add_node(component=retriever, name="retriever", inputs=["Query"])
pipe.add_node(component=prompt_node, name="prompt_node", inputs=["retriever"])

In [ ]:
query = "who is father of Arya Stark" # input()
output = pipe.run(query=query)

print(output["answers"][0].answer)
